# Calculate the state and week of the year based on coordinate and date

Step in OpenRefine before importing in R

Facet by issue
create new column BRI
Facet BRI (Basis of Record Invalid), choose blank
Facet by CountryCode, choose US
#Step only for Asclepias:
Facet by name, choose Asclepias incarnata, syriaca, tuberosa, verticillata, fascicularis, asperula, viridis, speciosa, curassavica

In [1]:
getwd()

[1] "/Users/vlemieuxlabonte"

Open data and remove rows with NA

In [2]:
Asc<-read.csv("asclepias_short.csv", header = T)
Asc_clean<-na.omit(Asc)
Dan<-read.csv("Danaus_short.csv",header = T)
Dan_clean<-na.omit(Dan)
head(Asc)
head(Dan)

name,long,lat,year,date
Asclepias curassavica,-82.44129,29.60351,2017,2017-01-26
Asclepias syriaca,-76.50381,39.57328,2017,2017-01-01
Asclepias fascicularis,-118.61500,34.27285,2017,2017-01-05
Asclepias curassavica,-80.16004,25.67369,2017,2017-01-16
Asclepias syriaca,-74.30058,40.68552,2017,2017-01-19
Asclepias curassavica,-122.35302,37.96350,2017,2017-01-06


name,long,lat,year,date
Danaus plexippus,-82.54852,27.34174,2017,2017-01-30
Danaus plexippus,-117.85259,34.13347,2017,2017-01-09
Danaus plexippus,-117.85254,34.13339,2017,2017-01-03
Danaus plexippus,-97.17492,26.14002,2017,2017-01-12
Danaus plexippus,-118.42388,34.01702,2017,2017-01-17
Danaus plexippus,-97.17505,26.13994,2017,2017-01-24


Open library for states determination

In [3]:
library(sp)
library(maps)
library(maptools)

Warning message:
“package ‘sp’ was built under R version 3.3.2”Warning message:
“package ‘maptools’ was built under R version 3.3.2”Checking rgeos availability: TRUE


Function to dertemine state based on geographic coordinates

In [4]:
latlong2state <- function(pointsDF) {
  # Prepare SpatialPolygons object with one SpatialPolygon
  # per state (plus DC, minus HI & AK)
  states <- map('state', fill=TRUE, col="transparent", plot=FALSE)
  IDs <- sapply(strsplit(states$names, ":"), function(x) x[1])
  states_sp <- map2SpatialPolygons(states, IDs=IDs,
                                   proj4string=CRS("+proj=longlat +datum=WGS84"))
  
  # Convert pointsDF to a SpatialPoints object 
  pointsSP <- SpatialPoints(pointsDF, 
                            proj4string=CRS("+proj=longlat +datum=WGS84"))
  
  # Use 'over' to get _indices_ of the Polygons object containing each point 
  indices <- over(pointsSP, states_sp)
  
  # Return the state names of the Polygons object containing each point
  stateNames <- sapply(states_sp@polygons, function(x) x@ID)
  stateNames[indices]}

Calculate state

In [5]:
coord_Asc<-data.frame(x=Asc_clean$long,y=Asc_clean$lat)
coord_Dan<-data.frame(x=Dan_clean$long,y=Dan_clean$lat)
states_Asc<-latlong2state(coord_Asc)
states_Dan<-latlong2state(coord_Dan)
head(states_Asc)
head(states_Dan)

[1] "florida"    "maryland"   "california" NA           "new jersey"
[6] "california"

[1] "florida"    "california" "california" NA           "california"
[6] NA

Calculate date

In [6]:
date_Asc<-as.Date(Asc_clean$date)
date_Dan<-as.Date(Dan_clean$date)
time_Asc<-data.frame(DOY=as.POSIXlt(date_Asc, format = "20%y-%m-%d")$yday+1,
           WOY=as.numeric(format(date_Asc, format = "%W"))+1)
time_Dan<-data.frame(DOY=as.POSIXlt(date_Dan, format = "20%y-%m-%d")$yday+1,
           WOY=as.numeric(format(date_Dan, format = "%W")))


Attach the new columns to the cleaned data frame

In [7]:
Asc_state_WOY<-cbind(Asc_clean,states_Asc,time_Asc)
Dan_state_WOY<-cbind(Dan_clean,states_Dan,time_Dan)

In [8]:
Asc_state_WOY_clean=na.omit(Asc_state_WOY)
Dan_state_WOY_clean=na.omit(Dan_state_WOY)

In [9]:
write.table(Asc_state_WOY_clean, "Asc_state_WOY_clean.csv", sep=",", row.names=FALSE, fileEncoding="UTF-8")
write.table(Dan_state_WOY_clean, "Dan_state_WOY_clean.csv", sep=",", row.names=FALSE, fileEncoding="UTF-8")

Export the data frames as CSV file

In [10]:
head(Asc_state_WOY_clean)
head(Dan_state_WOY_clean)

,name,long,lat,year,date,states_Asc,DOY,WOY
1,Asclepias curassavica,-82.44129,29.60351,2017,2017-01-26,florida,26,5
2,Asclepias syriaca,-76.50381,39.57328,2017,2017-01-01,maryland,1,1
3,Asclepias fascicularis,-118.61500,34.27285,2017,2017-01-05,california,5,2
5,Asclepias syriaca,-74.30058,40.68552,2017,2017-01-19,new jersey,19,4
6,Asclepias curassavica,-122.35302,37.96350,2017,2017-01-06,california,6,2
7,Asclepias syriaca,-88.10802,41.71613,2017,2017-01-13,illinois,13,3


,name,long,lat,year,date,states_Dan,DOY,WOY
1,Danaus plexippus,-82.54852,27.34174,2017,2017-01-30,florida,30,5
2,Danaus plexippus,-117.85259,34.13347,2017,2017-01-09,california,9,2
3,Danaus plexippus,-117.85254,34.13339,2017,2017-01-03,california,3,1
5,Danaus plexippus,-118.42388,34.01702,2017,2017-01-17,california,17,3
7,Danaus plexippus,-122.05990,36.98272,2017,2017-01-28,california,28,4
8,Danaus plexippus,-117.85247,34.13336,2017,2017-01-09,california,9,2
